In [1]:
from neo4j import GraphDatabase

# Connect to Neo4j
uri = "bolt://localhost:7687"  # Change this if on Databricks or another host
username = "neo4j"
password = "12345678"

driver = GraphDatabase.driver(uri, auth=(username, password))

# === User Data ===
users = [
    {"user_id": "U1", "username": "social_star", "join_date": "2020-01-15", "follower_count": 12500},
    {"user_id": "U2", "username": "tech_guru", "join_date": "2019-05-22", "follower_count": 8700},
    {"user_id": "U3", "username": "fitness_fan", "join_date": "2021-03-10", "follower_count": 4300},
    {"user_id": "U4", "username": "data_dude", "join_date": "2018-11-18", "follower_count": 18000},
    {"user_id": "U5", "username": "gadget_queen", "join_date": "2020-09-05", "follower_count": 3000},
    {"user_id": "U6", "username": "health_blogger", "join_date": "2021-02-14", "follower_count": 2100},
    {"user_id": "U7", "username": "eco_activist", "join_date": "2019-08-22", "follower_count": 10000},
    {"user_id": "U8", "username": "fashionista", "join_date": "2020-04-28", "follower_count": 8000},
    {"user_id": "U9", "username": "travel_enthusiast", "join_date": "2022-06-12", "follower_count": 5400},
    {"user_id": "U10", "username": "bookworm", "join_date": "2021-12-07", "follower_count": 6700},
]

# === Create User Nodes ===
def create_users(tx, users):
    for user in users:
        tx.run("""
            MERGE (u:User {user_id: $user_id})
            SET u.username = $username,
                u.join_date = date($join_date),
                u.follower_count = $follower_count
        """, **user)

# === Run it ===
with driver.session() as session:
    session.write_transaction(create_users, users)

print("✅ 10 users created in Neo4j.")

C:\Users\MaisoonAboFakher\AppData\Local\Temp\ipykernel_35880\3352012856.py:36: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_users, users)
Transaction failed and will be retried in 0.8152345703746582s (Couldn't connect to localhost:7687 (resolved to ('[::1]:7687', '127.0.0.1:7687')):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [WinError 10061] No connection could be made because the target machine actively refused it)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [WinError 10061] No connection could be made because the target machine actively refused it))
Transaction failed and will be retried in 2.2660678707479516s (Couldn't connect to localhost:7687 (resolved to ('[::1]:7687', '127.0.0.1:7687')):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [WinError 10061] No connection could be made because the target machine 

ServiceUnavailable: Couldn't connect to localhost:7687 (resolved to ('[::1]:7687', '127.0.0.1:7687')):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [WinError 10061] No connection could be made because the target machine actively refused it)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [WinError 10061] No connection could be made because the target machine actively refused it)

### Create Users (10 Users)

In [ ]:
# --- Post Data --- #
posts = [
    {"post_id": "P1", "timestamp": "2023-01-05T09:30:00", "content_type": "image", "like_count": 453},
    {"post_id": "P2", "timestamp": "2023-01-06T14:22:00", "content_type": "text", "like_count": 127},
    {"post_id": "P3", "timestamp": "2023-01-07T18:15:00", "content_type": "video", "like_count": 2145},
    {"post_id": "P4", "timestamp": "2023-01-08T11:30:00", "content_type": "text", "like_count": 987},
    {"post_id": "P5", "timestamp": "2023-01-09T13:00:00", "content_type": "image", "like_count": 203},
    {"post_id": "P6", "timestamp": "2023-01-10T15:30:00", "content_type": "video", "like_count": 1145},
    {"post_id": "P7", "timestamp": "2023-01-11T17:00:00", "content_type": "text", "like_count": 55},
    {"post_id": "P8", "timestamp": "2023-01-12T12:45:00", "content_type": "image", "like_count": 322},
    {"post_id": "P9", "timestamp": "2023-01-13T10:30:00", "content_type": "video", "like_count": 783},
    {"post_id": "P10", "timestamp": "2023-01-14T14:00:00", "content_type": "text", "like_count": 150},
    {"post_id": "P11", "timestamp": "2023-01-15T16:00:00", "content_type": "video", "like_count": 952},
    {"post_id": "P12", "timestamp": "2023-01-16T19:15:00", "content_type": "image", "like_count": 647},
    {"post_id": "P13", "timestamp": "2023-01-17T10:30:00", "content_type": "text", "like_count": 302},
    {"post_id": "P14", "timestamp": "2023-01-18T11:45:00", "content_type": "video", "like_count": 1280},
    {"post_id": "P15", "timestamp": "2023-01-19T13:00:00", "content_type": "image", "like_count": 200},
    {"post_id": "P16", "timestamp": "2023-01-20T14:15:00", "content_type": "text", "like_count": 75},
    {"post_id": "P17", "timestamp": "2023-01-21T15:30:00", "content_type": "video", "like_count": 56},
    {"post_id": "P18", "timestamp": "2023-01-22T17:00:00", "content_type": "image", "like_count": 300},
    {"post_id": "P19", "timestamp": "2023-01-23T18:15:00", "content_type": "text", "like_count": 190},
    {"post_id": "P20", "timestamp": "2023-01-24T19:00:00", "content_type": "video", "like_count": 245}
]

# --- Function to Insert Posts --- #
def create_posts(tx, posts):
    for post in posts:
        tx.run("""
            MERGE (p:Post {post_id: $post_id})
            SET p.timestamp = datetime($timestamp),
                p.content_type = $content_type,
                p.like_count = $like_count
        """, **post)

# --- Execute --- #
with driver.session() as session:
    session.write_transaction(create_posts, posts)

print("✅ 20 posts created in Neo4j.")